# Callbacks beyond task generation: E-Mail

While we typically use callbacks in **autoSTED** to enqueue new acquisition tasks, we can do a lot more since they are just Python functions (or callable objects). 

Here, we build a function to **send a progress report via e-mail** (listing how many images have been acquired) and include it as a callback into an acquisiton pipeline.

In [ ]:
from smtplib import SMTP
from email.message import EmailMessage
from getpass import getpass
from textwrap import dedent
from time import strftime, localtime, time

from autosted.pipeline import AcquisitionPipeline


def send_email(server, sender, password, message_content, recipient=None, subject=None, username=None):

    # Build message
    msg = EmailMessage()
    msg.set_content(message_content)
    msg["Subject"] = subject # may be None for no subject
    msg["From"] = sender
    # if no recipient is specified, send to yourself
    msg["To"] = sender if recipient is None else recipient

    # connect to SMTP server (with TLS) and send
    # NOTE: works e.g. with Gmail, but you may need an extra App password:
    # https://support.google.com/accounts/answer/185833?hl=en
    with SMTP(server, 587) as s:
        s.ehlo()
        s.starttls()
        s.ehlo()
        # login with sender email or extra username if given
        s.login(sender if username is None else username, password)
        s.send_message(msg)


def get_number_of_acquisitions_per_level(pipeline=None):
    # get currently running pipeline if none given
    if pipeline is None:
        pipeline = AcquisitionPipeline.running_instance
    # count number of acquisitions in pipeline's data for each hierarchy level
    acquisition_numbers = {}
    for level in pipeline.hierarchy_levels:
        num_acquisitions_at_level = len([idx for idx in pipeline.data.keys() if idx[-1][0] == level])
        acquisition_numbers[level] = num_acquisitions_at_level
    return acquisition_numbers


def make_progress_report(acquisition_numbers):
    report = dedent(
    f"""
    autoSTED status report at {strftime("%d.%m.%Y %H:%M:%S", localtime(time()))}
    ----------------------
    
    Number of acquisitions:
    """
    )
    for level, num_acquisitions_at_level in acquisition_numbers.items():
        report += f"{level}: {num_acquisitions_at_level}\n"
    report += f"Total acquisitions: {sum(acquisition_numbers.values())}\n"
    return report


In [ ]:
# test progress report generation
print(make_progress_report({"overview": 12, "detail": 24}))

## A progress report callback

Here, we define a function ```send_progress_report``` that will get the current number of acquisitions of the running pipeline and send an e-mail with a progress report every 100 acquiistions.

**NOTE:** Since we will use this function like other callbacks in an autoSTED pipeline, make sure it returns None (otherwise, the pipeline will try to interpret return value as new acquisition tasks).

**WARNING:** to send an e-mail you need to log into the SMTP server of your provider and need to enter your password, so be careful to limit access of others to this notebook while it is running

In [ ]:
# get password via hidden prompt
# NOTE: you could just write it as a string here, but writing password in plain text is a bad idea
# also NOTE: since the password is saved in memory here,
# others with access to this notebook may be able to read it while the notebook is running
password = getpass()
server = 'smtp.gmail.com'
sender = 'my-email@gmail.com'

def send_progress_report(every_n_acquisitions=100):
    acquisition_numbers = get_number_of_acquisitions_per_level()
    total_acquisitions = sum(acquisition_numbers.values())

    if total_acquisitions % every_n_acquisitions == 0:
        message_content = make_progress_report(acquisition_numbers)
        send_email(server, sender, password, message_content)

## Including report sending in a pipeline

Now, we build an Acquisition pipeline similar to ```overview_spiral.ipynb``` and include the reporting functionality:

In [ ]:
from autosted.callback_buildingblocks import (
    FOVSettingsGenerator,
    JSONSettingsLoader,
    LocationRemover,
    SpiralOffsetGenerator,
)
from autosted.imspector import get_current_stage_coords
from autosted.stoppingcriteria import MaximumAcquisitionsStoppingCriterion
from autosted.taskgeneration import AcquisitionTaskGenerator

In [ ]:
# where to save & whether to save combined HDF5 file
save_folder = "acquisition_data/spiral-test"
save_hdf5 = False

# path of measurement parameters (dumped to JSON file)
measurement_parameters = "config_json/20241010_overview_3d_640.json"

# yx move size between images in spiral
move_size = [50e-6, 50e-6]

In [ ]:
# get current coordinates and print, so we can go back to that position
start_coords = get_current_stage_coords()
print(start_coords)

Finally, we can build and run the pipeline.

- we attach our ```send_progress_report``` as a second callback at the "field" level (in addition the the ```next_position_generator``` callback)
- at the end of the run, we send an e-mail as well

In [ ]:
# build pipeline object (1 level: 'field')
pipeline = AcquisitionPipeline(
    save_folder, ["field"], save_combined_hdf5=save_hdf5
)

# callback that will create an acquisition task with given measurement parameters
# at the next stage coordinates in the coordinate list (the next 'position')
next_position_generator = AcquisitionTaskGenerator(
    "field",
    # 1. load basic measurement parameters from file
    LocationRemover(JSONSettingsLoader(measurement_parameters)),
    # 2. (optional) update FOV to match spiral move size in yx (leave z & pixel size as-is -> None)
    FOVSettingsGenerator(lengths=[None] + move_size, pixel_sizes=None),
    # 3. get next position in spiral
    SpiralOffsetGenerator(move_size, start_coords[1:])
)

# attach callback so that after each position, the next one will be enqueued
pipeline.add_callback(next_position_generator, "field")

# NOTE: attach the function to send progress report as a callback
pipeline.add_callback(send_progress_report, "field")

# set maximum number of acquisitions before stop
pipeline.add_stopping_condition(MaximumAcquisitionsStoppingCriterion(500))

# start with initial task from callback
pipeline.run(next_position_generator)

# NOTE: send a mail once the pipeline has finished
finish_message = f"autoSTED pipeline finished at {strftime('%d.%m.%Y %H:%M:%S', localtime(time()))}"
send_email(server, sender, password, finish_message)